In [1]:
# flask
from flask import (
    Flask,
    request,
    abort
)

In [2]:
# Linebot-sdk
from linebot import ( 
    LineBotApi,
    WebhookHandler
)

from linebot.exceptions import ( 
    InvalidSignatureError
)

from linebot.models.events import (
    FollowEvent,
    MessageEvent,
    PostbackEvent
)

from linebot.models.messages import (
    TextMessage,
    ImageMessage
    
)

from linebot.models.send_messages import (
    TextSendMessage,
    ImageSendMessage
)

In [3]:
# Other utile modules
from pprint import pprint
import json
import requests
from pymongo import MongoClient, ReturnDocument
from googletrans import Translator, LANGUAGES, LANGCODES

In [4]:
# load necessary Linebot information to use LineBotApi
with open("../line_secret_key", "r", encoding="utf-8") as infile:
    conf_json = json.load(infile)
    
channel_access_token = conf_json.get("channel_access_token") # 回傳訊息給 Line 時使用
secret_key           = conf_json.get("secret_key") # WebhookHandler驗證事件消息是否來自Line使用
linebot_developer_id = conf_json.get("self_user_id")
server_url           = "https://" + conf_json.get("server_url")
rich_menu_id         = conf_json.get("rich_menu_id")

# load mongodb settings
with open("../mongo_client_settings", "r", encoding="utf-8") as infile:
    mongo_client_settings = json.loads(infile.read())

In [5]:
line_bot_api = LineBotApi(channel_access_token) # 回傳訊息給 Line 時使用
handler = WebhookHandler(secret_key) # WebhookHandler驗證事件消息是否來自Line使用

In [6]:
app = Flask(__name__, static_url_path="/res", static_folder="../res")

In [7]:
# Line MessegeEvent entrypoint
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)
    app.logger.info("Request body: " + body)
    
    # Line webhook url verify
    if eval(body).get("events")[0].get("source").get("userId") == "Udeadbeefdeadbeefdeadbeefdeadbeef":
        return "OK"

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [8]:
follow_message_list = [
    TextSendMessage("Welcome to my translator bot"),
    TextSendMessage("This bot would translate Mandarin to English by default")
]

# handle FollowEvent
"""
1. Get user info and save it
2. Bind function UI to the user
3. Send "Welcome" messages to the user

"""
@handler.add(FollowEvent)
def follows(event):
    
    # Get user info and save it
    user_profile = line_bot_api.get_profile(event.source.user_id)
    user_profile = user_profile.as_json_dict()
    user_profile["src"] = "auto"
    user_profile["dest"] = "en"
    pprint(user_profile)
#     print(type(user_profile))
#     pprint(user_profile.__doc__)
#     with open("../users.json", "a", encoding="utf-8") as outfile:
#         outfile.write(json.dumps(user_profile))
#         outfile.write("\n")
    uri = mongo_client_settings.get("uri")
    port = mongo_client_settings.get("port")
    conn = MongoClient(uri, port=port)
    db = conn.translator
    collection = db.translator_users
    try:
        inserted_user_id = collection.insert_one(user_profile).inserted_id
    except errors.InvalidDocument as inv_doc_err:
        print(inv_doc_err)
    
                     
    # Bind function UI to the user
    bind_rich_menu_base_url = 'https://api.line.me/v2/bot/user/%s/richmenu/%s'
    bind_rich_menu_id = rich_menu_id
                     
    bind_rich_menu_id_upload_endpoint = bind_rich_menu_base_url % (event.source.user_id, bind_rich_menu_id)
    bind_rich_menu_request_headers = {'Content-Type':'image/jpeg',
                                      'Authorization':'Bearer %s' % channel_access_token
                                     }
    
    bind_rich_menu_response = requests.post(bind_rich_menu_id_upload_endpoint,
                                            headers=bind_rich_menu_request_headers)
    
                     
    # Send "Welcome" messages
    line_bot_api.reply_message(event.reply_token,
                               follow_message_list)
    
    

In [9]:
LANGUAGES

{'af': 'afrikaans',
 'sq': 'albanian',
 'am': 'amharic',
 'ar': 'arabic',
 'hy': 'armenian',
 'az': 'azerbaijani',
 'eu': 'basque',
 'be': 'belarusian',
 'bn': 'bengali',
 'bs': 'bosnian',
 'bg': 'bulgarian',
 'ca': 'catalan',
 'ceb': 'cebuano',
 'ny': 'chichewa',
 'zh-cn': 'chinese (simplified)',
 'zh-tw': 'chinese (traditional)',
 'co': 'corsican',
 'hr': 'croatian',
 'cs': 'czech',
 'da': 'danish',
 'nl': 'dutch',
 'en': 'english',
 'eo': 'esperanto',
 'et': 'estonian',
 'tl': 'filipino',
 'fi': 'finnish',
 'fr': 'french',
 'fy': 'frisian',
 'gl': 'galician',
 'ka': 'georgian',
 'de': 'german',
 'el': 'greek',
 'gu': 'gujarati',
 'ht': 'haitian creole',
 'ha': 'hausa',
 'haw': 'hawaiian',
 'iw': 'hebrew',
 'hi': 'hindi',
 'hmn': 'hmong',
 'hu': 'hungarian',
 'is': 'icelandic',
 'ig': 'igbo',
 'id': 'indonesian',
 'ga': 'irish',
 'it': 'italian',
 'ja': 'japanese',
 'jw': 'javanese',
 'kn': 'kannada',
 'kk': 'kazakh',
 'km': 'khmer',
 'ko': 'korean',
 'ku': 'kurdish (kurmanji)',
 'ky

In [ ]:
# LANGCODES = dict(map(reversed, LANGUAGES.items()))

In [10]:
LANGCODES

{'afrikaans': 'af',
 'albanian': 'sq',
 'amharic': 'am',
 'arabic': 'ar',
 'armenian': 'hy',
 'azerbaijani': 'az',
 'basque': 'eu',
 'belarusian': 'be',
 'bengali': 'bn',
 'bosnian': 'bs',
 'bulgarian': 'bg',
 'catalan': 'ca',
 'cebuano': 'ceb',
 'chichewa': 'ny',
 'chinese (simplified)': 'zh-cn',
 'chinese (traditional)': 'zh-tw',
 'corsican': 'co',
 'croatian': 'hr',
 'czech': 'cs',
 'danish': 'da',
 'dutch': 'nl',
 'english': 'en',
 'esperanto': 'eo',
 'estonian': 'et',
 'filipino': 'tl',
 'finnish': 'fi',
 'french': 'fr',
 'frisian': 'fy',
 'galician': 'gl',
 'georgian': 'ka',
 'german': 'de',
 'greek': 'el',
 'gujarati': 'gu',
 'haitian creole': 'ht',
 'hausa': 'ha',
 'hawaiian': 'haw',
 'hebrew': 'iw',
 'hindi': 'hi',
 'hmong': 'hmn',
 'hungarian': 'hu',
 'icelandic': 'is',
 'igbo': 'ig',
 'indonesian': 'id',
 'irish': 'ga',
 'italian': 'it',
 'japanese': 'ja',
 'javanese': 'jw',
 'kannada': 'kn',
 'kazakh': 'kk',
 'khmer': 'km',
 'korean': 'ko',
 'kurdish (kurmanji)': 'ku',
 'ky

In [11]:
LANGCODES["chinese_tw"] = LANGCODES.pop("chinese (traditional)")
LANGCODES["chinese_cn"] = LANGCODES.pop("chinese (simplified)")

In [ ]:
@handler.add(MessageEvent, message=TextMessage)
def translate(event):
    
    def getUserTranslationSettings(user_profile):
        with open("../users.json", "r", encoding="utf-8") as infile:
            user_list = [eval(u) for u in infile.read().strip().split(sep="\n")]
            for user in user_list:
                if (user.get("userId") == user_profile.get("userId")) and (user.get("src") != None):
                    user_profile = user # 會拿到最新的設定
        return user_profile
    
    def _changeUserTranslationDest(user_profile, dest):       
        with open("../users.json", "r", encoding="utf-8") as infile:
            user_list = [eval(u) for u in infile.read().strip().split(sep="\n")]
            for user in user_list:
                if user.get("userId") == user_profile.get("userId") and user.get("src") != None:
                    user_profile = user
        with open("../users.json", "a", encoding="utf-8") as outfile:
            user_profile["dest"] = dest
            outfile.write(repr(user_profile)+"\n")
            
    
    def checkIfChangeSettings():
#         print(type(event.message.text.find("^to_")))
#         print(event.message.text.find("^to_"))
        if event.message.text.find("^to_") != -1: # find("pattern")回傳值型態為 int， 若有這個字串，回傳0：沒有這個字串則回傳 -1
            lang = event.message.text[4:]
            dest = LANGCODES.get(lang)
            
            _changeUserTranslationDest(user_profile, dest=dest)
        else:
            print("no change")
        
        return getUserTranslationSettings(user_profile)
            
                  
#     print(type(event))
#     print(event)
#     print(type(event.message.text.find("^to_")))
    user_profile = line_bot_api.get_profile(event.source.user_id).as_json_dict()
    user_profile = checkIfChangeSettings()
    
#     print(type(user_profile))
#     print(user_profile)
#     user_profile = getUserTranslationSettings(user_profile)
#     print(type(user_profile))
#     print(user_profile)
    
       
#     if event.message.text.find("\to_english"):
#         changeUserTranslationDest(user_profile, dest="en")
        
    
#     if event.message.text.find("\to_french"):
#         changeUserTranslationDest(user_profile, dest="fr")
        
#     user_profile = getUserTranslationSettings(user_profile)

           
    translator = Translator(service_urls=['translate.google.com',
                                          'translate.google.com.tw',
                                          'translate.google.co.kr',
                                          'translate.google.co.jp'
                                         ]
                           )
#     user_profile = getUserTranslationSettings(user_profile)
    
    dest = user_profile.get("dest")
#     dest = "en"
        
    translated = TextSendMessage(translator.translate(event.message.text, dest=dest).text)

    
    line_bot_api.reply_message(event.reply_token,
                               translated
                              )
    

In [ ]:
with open("../mongo_client_settings", "r", encoding="utf-8") as infile:
    mongo_client_settings = json.loads(infile.read())

In [ ]:
mongo_client_settings.get("uri")

In [ ]:
# 接 mongodb
@handler.add(MessageEvent, message=TextMessage)
def translate(event):
    """
    step1. 建立Mongodb連線
    step2. 檢查是否要改翻譯目標語言，並把用戶的翻譯設定撈出來
    step3. 呼叫 googletrans API 並把翻譯內容回傳使用者
    
    """
    
    def _getUserTranslationSettings(user_profile):
        user_profile = collection.find_one({"userId":userId})
        return user_profile
    
    def _changeUserTranslationDest(user_profile, dest):  
        user_lang_modified = collection.find_one_and_update(filter={"userId":userId},
                                                            update={"$set":{"dest":dest}}, 
                                                            return_document=ReturnDocument.AFTER)
        return user_lang_modified
            
    
    def checkIfChangeSettings():
        if event.message.text.find("^to_") != -1: # find("pattern")回傳值型態為 int， 若有這個字串，回傳0：沒有這個字串則回傳 -1
            lang = event.message.text[4:]
            dest = LANGCODES.get(lang)
            user_lang_modified = _changeUserTranslationDest(user_profile, dest=dest)
            print("Translation destination language is successfully changed to: %s" % user_lang_modified.get("dest"))
            return user_lang_modified
        
        else:
            print("no change detected.")
            return _getUserTranslationSettings(user_profile)
    
    
    
    # step1
    with open("../mongo_client_settings", "r", encoding="utf-8") as infile:
        mongo_client_settings = json.loads(infile.read())
    
    uri = mongo_client_settings.get("uri")
    port = mongo_client_settings.get("port")
    conn = MongoClient(uri, port=port)
    db = conn.translator
    collection = db.translator_users
    
    
    # step2             
    user_profile = line_bot_api.get_profile(event.source.user_id).as_json_dict()  
    userId = user_profile.get("userId")
    user_profile = checkIfChangeSettings() # return the user_profile with "src" and "dest"
    
    
    # step3
    translator = Translator(service_urls=['translate.google.com',
                                          'translate.google.com.tw',
                                          'translate.google.co.kr',
                                          'translate.google.co.jp'
                                         ]
                           )
                        
    
    dest = user_profile.get("dest")
                            
    translated_text_send_message = TextSendMessage(translator.translate(event.message.text, dest=dest).text)

    
    line_bot_api.reply_message(event.reply_token,
                               translated_text_send_message
                              )

In [13]:
# PostbackEvent
@handler.add(PostbackEvent, message=TextMessage)
def translate(event):
    """
    step1. 建立Mongodb連線
    step2. 檢查是否要改翻譯目標語言，並把用戶的翻譯設定撈出來
    step3. 呼叫 googletrans API 並把翻譯內容回傳使用者
    
    """
    
    def _getUserTranslationSettings(user_profile):
        user_profile = collection.find_one({"userId":userId})
        return user_profile
    
    def _changeUserTranslationDest(user_profile, dest):  
        user_lang_modified = collection.find_one_and_update(filter={"userId":userId},
                                                            update={"$set":{"dest":dest}}, 
                                                            return_document=ReturnDocument.AFTER)
        return user_lang_modified
            
    
    def checkIfChangeSettings():
        if event.message.text.find("^to_") != -1: # find("pattern")回傳值型態為 int， 若有這個字串，回傳0：沒有這個字串則回傳 -1
            lang = event.message.text[4:]
            dest = LANGCODES.get(lang)
            user_lang_modified = _changeUserTranslationDest(user_profile, dest=dest)
            print("Translation destination language is successfully changed to: %s" % user_lang_modified.get("dest"))
            return user_lang_modified
        
        else:
            print("no change detected.")
            return _getUserTranslationSettings(user_profile)
    
    
    user_profile = line_bot_api.get_profile(event.source.user_id).as_json_dict()
    print(type(user_profile))
    print(user_profile)
#     # step1
#     with open("../mongo_client_settings", "r", encoding="utf-8") as infile:
#         mongo_client_settings = json.loads(infile.read())
    
#     uri = mongo_client_settings.get("uri")
#     port = mongo_client_settings.get("port")
#     conn = MongoClient(uri, port=port)
#     db = conn.translator
#     collection = db.translator_users
    
    
#     # step2             
#     user_profile = line_bot_api.get_profile(event.source.user_id).as_json_dict()  
#     userId = user_profile.get("userId")
#     user_profile = checkIfChangeSettings() # return the user_profile with "src" and "dest"
    
    
#     # step3
#     translator = Translator(service_urls=['translate.google.com',
#                                           'translate.google.com.tw',
#                                           'translate.google.co.kr',
#                                           'translate.google.co.jp'
#                                          ]
#                            )
                        
    
#     dest = user_profile.get("dest")
                            
#     translated_text_send_message = TextSendMessage(translator.translate(event.message.text, dest=dest).text)

    
#     line_bot_api.reply_message(event.reply_token,
#                                translated_text_send_message
#                               )

In [ ]:
if __name__ == "__main__":
    app.run(host="0.0.0.0")

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


{"events":[{"type":"message","replyToken":"c8ffd21ece994a3e8733cb4d892647e6","source":{"userId":"U60d8a876f6837d1af4cac912adb1ee02","type":"user"},"timestamp":1552698988487,"message":{"type":"text","id":"9521950423938","text":"翻成英文"}}],"destination":"Ue3bcbd0b951f367dd9a6bfc1a0d31e3e"}
no change detected.


192.168.112.5 - - [16/Mar/2019 01:16:49] "POST / HTTP/1.1" 200 -
192.168.112.5 - - [16/Mar/2019 01:16:50] "POST / HTTP/1.1" 200 -


{"events":[{"type":"postback","replyToken":"c014c734dbc54486b9c173c5f0b98318","source":{"userId":"U60d8a876f6837d1af4cac912adb1ee02","type":"user"},"timestamp":1552698988489,"postback":{"data":"^to_english"}}],"destination":"Ue3bcbd0b951f367dd9a6bfc1a0d31e3e"}
{"events":[{"type":"message","replyToken":"dd03cd60e56b4aca8d9faee11efda7b2","source":{"userId":"U60d8a876f6837d1af4cac912adb1ee02","type":"user"},"timestamp":1552958232350,"message":{"type":"text","id":"9538990214502","text":"Hello"}}],"destination":"Ue3bcbd0b951f367dd9a6bfc1a0d31e3e"}
no change detected.


192.168.112.5 - - [19/Mar/2019 01:17:37] "POST / HTTP/1.1" 200 -
192.168.112.5 - - [19/Mar/2019 01:17:45] "POST / HTTP/1.1" 200 -


...

192.168.112.5 - - [19/Mar/2019 01:19:54] "POST / HTTP/1.1" 200 -


{"events":[{"type":"message","replyToken":"436071cd197047cdaf02f21112fdde75","source":{"userId":"U60d8a876f6837d1af4cac912adb1ee02","type":"user"},"timestamp":1552958382270,"message":{"type":"text","id":"9538999717837","text":"I am tired today"}}],"destination":"Ue3bcbd0b951f367dd9a6bfc1a0d31e3e"}
no change detected.


192.168.112.5 - - [19/Mar/2019 01:20:03] "POST / HTTP/1.1" 200 -


{"events":[{"type":"message","replyToken":"d79684b4d83a4d60a32ad2f5f75cd616","source":{"userId":"U60d8a876f6837d1af4cac912adb1ee02","type":"user"},"timestamp":1552969010208,"message":{"type":"text","id":"9539763046675","text":"I am tired"}}],"destination":"Ue3bcbd0b951f367dd9a6bfc1a0d31e3e"}
no change detected.


192.168.112.5 - - [19/Mar/2019 04:17:11] "POST / HTTP/1.1" 200 -


In [ ]:
test = {"displayName": "Even(Yves)", "pictureUrl": "https://profile.line-scdn.net/0hKUsnnZZSFHpOFzv9_lNrLXJSGhc5ORIyNiNfTGoSSB5lclIvdHldH24VHx00JAYsdXcMS24UTB5k", "userId": "U60d8a876f6837d1af4cac912adb1ee02"}

In [ ]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
    "[::text:]翻成英文":text_message_to_en,
    "[::text:]翻成法文":text_message_to_fr,
#     "[::flex:]促銷食材":flexCarouselSendMeesage,
    "[::flex:]翻成日文":image_message,
    "[::text:]有什麼語言":image_message3,
    "[::text:]more":image_message3,
    "[::text:]Modify":image_message4,
    "[::text:]tryLiff":image_message3
}

In [ ]:
with open("../users.json", "r", encoding="utf-8") as infile:
    tmp_str = infile.read()
    tmp_list = tmp_str.split(sep="\n")
    pprint(tmp_list)

In [ ]:
tmp.split(sep="\n")

In [ ]:
line_bot_api.__dir__()